In [9]:
# PyPDF2
from PyPDF2 import PdfReader
import pandas as pd

In [10]:
reader = PdfReader("./resume_pdf/cv_sophal.pdf")
number_of_pages = len(reader.pages)
page = reader.pages[0]
text = page.extract_text()

In [11]:
number_of_pages

2

In [12]:
page

{'/Type': '/Page',
 '/Parent': {'/Type': '/Pages',
  '/MediaBox': [0, 0, 595, 842],
  '/Count': 2,
  '/Kids': [IndirectObject(1, 0, 5898121424),
   IndirectObject(25, 0, 5898121424)]},
 '/Resources': {'/ProcSet': ['/PDF', '/Text', '/ImageB', '/ImageC', '/ImageI'],
  '/ColorSpace': {'/Cs1': ['/ICCBased', IndirectObject(24, 0, 5898121424)]},
  '/Font': {'/TT2': {'/Type': '/Font',
    '/Subtype': '/TrueType',
    '/BaseFont': '/AAAAAC+Calibri',
    '/FontDescriptor': {'/Type': '/FontDescriptor',
     '/FontName': '/AAAAAC+Calibri',
     '/Flags': 4,
     '/FontBBox': [-503, -313, 1240, 1026],
     '/ItalicAngle': 0,
     '/Ascent': 952,
     '/Descent': -269,
     '/CapHeight': 632,
     '/StemV': 0,
     '/XHeight': 464,
     '/AvgWidth': 521,
     '/MaxWidth': 1328,
     '/FontFile2': {'/Length1': 39060, '/Filter': '/FlateDecode'}},
    '/ToUnicode': {'/Filter': '/FlateDecode'},
    '/FirstChar': 33,
    '/LastChar': 107,
    '/Widths': [517,
     579,
     631,
     488,
     226,
    

In [13]:
text

'PAGE 1  Thear Sophal          Address  : Str.289, Sangkat Boeung Kak 2, Khan Toul Kouk, Phnom Penh       Contact  : +855 81 591 594 / LinkedIn       Email  : sophal.thear@cadt.edu.kh / thearsophaltsp@gmail.com    Objective Apply for Data Science Researcher Personal  Information Gender             :  Male Nationality      :  Cambodian Date of Birth   :  24th July 1999 Place of Birth  :  Phnom Penh, Cambodia   Internship, Job, and Other Experiences  August/2023 – Present Data Science Researcher at Cambodia Academy of Digital Technology, Phnom Penh, Cambodia Responsibilities: • Digital Essential Skill Data Analysis Project • Resume Ranking Base on Job Description Project • Teaching Assistant of Information Visualization Course  February/2023 – July/2023 (Research Project) Predicting Proteins Involved in Secretion Systems: A Large, Imbalanced Supervised Learning Problem at TIMC, Grenoble, France Responsibilities: • Study statistic information about protein data (~84 millions proteins) • C

In [14]:
split_text = text.split('\n')
split_text

['PAGE 1  Thear Sophal          Address  : Str.289, Sangkat Boeung Kak 2, Khan Toul Kouk, Phnom Penh       Contact  : +855 81 591 594 / LinkedIn       Email  : sophal.thear@cadt.edu.kh / thearsophaltsp@gmail.com    Objective Apply for Data Science Researcher Personal  Information Gender             :  Male Nationality      :  Cambodian Date of Birth   :  24th July 1999 Place of Birth  :  Phnom Penh, Cambodia   Internship, Job, and Other Experiences  August/2023 – Present Data Science Researcher at Cambodia Academy of Digital Technology, Phnom Penh, Cambodia Responsibilities: • Digital Essential Skill Data Analysis Project • Resume Ranking Base on Job Description Project • Teaching Assistant of Information Visualization Course  February/2023 – July/2023 (Research Project) Predicting Proteins Involved in Secretion Systems: A Large, Imbalanced Supervised Learning Problem at TIMC, Grenoble, France Responsibilities: • Study statistic information about protein data (~84 millions proteins) • 